World Map Geovisualization using Folium and Covid data

In [88]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [89]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers={}
conn.request("GET","/summary",payload,headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

In [90]:
covid1 = json.loads(data)

In [91]:
pd.json_normalize(covid1['Countries'],sep=',')
df = pd.DataFrame(covid1['Countries'])

In [92]:
covid2 = df.drop(columns=['CountryCode','Slug','Date','Premium'],axis=1)
covid2

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,96,39799,4,1477,50,33114
1,Albania,168,15399,4,420,94,9500
2,Algeria,132,53072,6,1801,0,37170
3,Andorra,0,2696,0,55,0,1814
4,Angola,120,6366,0,218,27,2743
...,...,...,...,...,...,...,...
183,Viet Nam,2,1109,0,35,0,1024
184,Western Sahara,0,10,0,1,0,8
185,Yemen,1,2052,0,595,0,1329
186,Zambia,43,15458,0,337,58,14599


In [93]:
# Worst performing countries wtr population

covid2_temp = covid2
covid2_temp["Death Ratio"] = covid2["TotalDeaths"]/covid2["TotalConfirmed"]
covid2_temp.sort_values(by="Death Ratio", ascending=False).head(20)[["Country", "Death Ratio"]]

,Country,Death Ratio
185,Yemen,0.289961
110,Mexico,0.102484
82,Italy,0.101890
184,Western Sahara,0.100000
50,Ecuador,0.082913
178,United Kingdom,0.070765
33,Chad,0.070552
16,Belgium,0.062807
57,Fiji,0.062500
160,Sudan,0.061142


In [94]:
m = folium.Map(titles="Stamen Terrain",min_zoom=1.5)
m

In [95]:
url = 'http://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [96]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=covid2,
    columns=['Country','TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color ='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Cases',
).add_to(m)
m

In [97]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

In [98]:
coordinates=pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates.head()

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra


In [99]:
covid_final=pd.merge(covid2,coordinates,on='Country')

In [100]:
def plotDot(point):
    folium.CircleMarker(location = [point.latitude,point.longitude],radius=5,weight=2,popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],fill_color='#000000').add_to(m)
    
covid_final.apply(plotDot,axis=1)  
m.fit_bounds(m.get_bounds())
m   
m1 = folium.Map(titles='StamenToner',min_zoom=2)    

In [101]:
deaths=covid_final['TotalDeaths'].astype(float)
lat=covid_final['latitude'].astype(float)
lon=covid_final['longitude'].astype(float)
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

In [102]:
def plotDot(point):
    folium.CircleMarker(location = [point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.TotalDeaths],
                        fill_color='#000000').add_to(m1)
covid_final.apply(plotDot,axis=1)
m1.fit_bounds(m1.get_bounds())
m1